# Задание на дипломный проект «Резервное копирование» первого блока «Основы языка программирования Python».

Возможна такая ситуация, что мы хотим показать друзьям фотографии из социальных сетей, но соц. сети могут быть недоступны по каким-либо причинам. Давайте защитимся от такого.
Нужно написать программу для резервного копирования фотографий с профиля(аватарок) пользователя vk в облачное хранилище Яндекс.Диск.
Для названий фотографий использовать количество лайков, если количество лайков одинаково, то добавить дату загрузки.
Информацию по сохраненным фотографиям сохранить в json-файл.

### Задание:

Нужно написать программу, которая будет:

1. Получать фотографии с профиля. Для этого нужно использовать метод photos.get.
2. Сохранять фотографии максимального размера(ширина/высота в пикселях) на Я.Диске.
3. Для имени фотографий использовать количество лайков.
4. Сохранять информацию по фотографиям в json-файл с результатами.

### Входные данные:

Пользователь вводит:

1. id пользователя vk;
2. токен с Полигона Яндекс.Диска. Важно: Токен публиковать в github не нужно!

### Выходные данные:

1. json-файл с информацией по файлу:

    [{
    "file_name": "34.jpg",
    "size": "z"
    }]

2. Измененный Я.диск, куда добавились фотографии.

### Обязательные требования к программе:

1. Использовать REST API Я.Диска и ключ, полученный с полигона.
2. Для загруженных фотографий нужно создать свою папку.
3. Сохранять указанное количество фотографий(по умолчанию 5) наибольшего размера (ширина/высота в пикселях) на Я.Диске
4. делать прогресс-бар или логирование для отслеживания процесса программы.
5. Код программы должен удовлетворять PEP8.

### Необязательные требования к программе:

1. Сохранять фотографии и из других альбомов.
2. Сохранять фотографии из других социальных сетей. Одноклассники и Инстаграмм
3. Сохранять фотографии на Google.Drive.

**Советы:**

1. Для тестирования можно использовать аккаунт https://vk.com/begemot_korovin
2. Токен для VK api: 958eb5d439726565e9333aa30e50e0f937ee432e927f0dbd541c541887d919a7c56f95c04217915c32008

**Решение:**

In [3]:
import requests
import tqdm
from http.client import responses
import datetime
import json

YATOKEN = ''
VKTOKEN = ''


class UsersVK:
    API_BASE_URL = 'https://api.vk.com/method/'

    def __init__(self, token: str, user_id, version='5.129'):
        self.types = {'s': 1, 'm': 2, 'x': 3, 'o': 4, 'p': 5, 'q': 6, 'r': 7, 'y': 8, 'z': 9, 'w': 10}
        self.vksite = 'https://vk.com/'
        self.token = token
        self.version = version
        self.user_id = user_id
        response = requests.get(self.API_BASE_URL + 'photos.get',
                                params={'owner_id': self.user_id, 'photo_sizes': 1, 'count': 5, 'album_id': 'profile',
                                        'extended': 1,
                                        'access_token': token, 'v': 5.129})

        photos = response.json()
        self.photos = photos['response']['items']

    def get_photos(self):
        result = []
        download_files = []
        print('загрузка фотографий:')
        for item in tqdm.tqdm(self.photos):
            max_size = 0
            download_link = {}
            for link in item['sizes']:
                if max_size > self.types.get(link['type']):
                    continue
                else:
                    max_size = self.types.get(link['type'])
                    download_link = link
            url1 = download_link['url']
            date = datetime.datetime.fromtimestamp(item['date'])
            name = str(date.strftime('%d-%m-%y')) + 'id' + str(item['id']) + '.jpg'
            result.append(name)
            download_files.append({'file_name': name, 'size': download_link['type']})
            responce = requests.get(url1)
            with open(name, 'wb') as f:
                f.write(responce.content)

        with open("data_file.json", "w") as write_file:
            json.dump(download_files, write_file)
        return result


class YaUploader:
    def __init__(self, token: str):
        self.TOKEN = {'Authorization': 'OAuth ' + token}
        self.HOST_API = 'https://cloud-api.yandex.net:443'
        self.UPLOAD_LINK = '/v1/disk/resources/upload'
        self.FILES_LIST = '/v1/disk/resources/files'
        self.FOLDER_CREATE = '/v1/disk/resources'
        self.FILES = files

    def upload(self):
        print('Загрузка файлов на Яндекс Диск')
        folder = self.create_folder()
        for item in tqdm.tqdm(self.FILES):
            path = folder + '/' + item
            upload_link = requests.get(self.HOST_API + self.UPLOAD_LINK, params={'path': path}, headers=self.TOKEN)
            if upload_link.status_code != requests.codes.ok:
                return f'\nОшибка при получении URL. Код: ' \
                       f'{upload_link.status_code} ({responses[upload_link.status_code]})'
            files = {'file': open(item, 'rb')}
            request = requests.put(upload_link.json()['href'], params={'path': item}, files=files)
            if not (200 <= request.status_code < 300):
                return print(f'\nОшибка при загрузке файла. Код: ' \
                       f'{request.status_code} ({responses[request.status_code]})')
        return print(f'\nФайы загружеы. Код: {request.status_code} ({responses[request.status_code]})')

    def create_folder(self, folder_name='test'):
        folder = requests.put(self.HOST_API + self.FOLDER_CREATE,
                              params={'path': folder_name}, headers=self.TOKEN)
        if not (200 <= folder.status_code < 300):
            return f'\nОшибка при при создании папки. Код: ' \
                   f'{folder.status_code} ({responses[folder.status_code]})'
        return folder_name

    def file_list (self):
        result = []
        request = requests.get(self.HOST_API + self.FILES_LIST, headers=self.TOKEN)
        if not (200 <= request.status_code < 300):
            return [f'\nОшибка при получении списка файлов. Код: '
                    f'{request.status_code} ({responses[request.status_code]})']
        result += [f'({x["name"]} ({str(int(x["size"] / 1024))} kB )' for x in request.json()['items']]
        return print(result)


user_1 = UsersVK(VKTOKEN, 552934290)
folder = 'test'
files = user_1.get_photos()
uploader = YaUploader(YATOKEN)
uploader.upload()
uploader.file_list()

  0%|          | 0/5 [00:00<?, ?it/s]

загрузка фотографий:


100%|██████████| 5/5 [00:03<00:00,  1.42it/s]


Загрузка файлов на Яндекс Диск


100%|██████████| 5/5 [00:24<00:00,  4.87s/it]



Файы загружеы. Код: 201 (Created)
['(12-07-19id457239018.jpg (27 kB )', '(05-07-20id457239023.jpg (272 kB )', '(06-07-20id457239027.jpg (163 kB )', '(06-07-20id457239029.jpg (163 kB )', '(06-07-20id457239031.jpg (220 kB )']
